In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [ ]:
url = 'https://www.thewhiskyexchange.com/c/40/single-malt-scotch-whisky'

In [ ]:
def soup(url):
  return BeautifulSoup(requests.get(url).content,'lxml')

In [ ]:
soup1 = soup(url)

In [ ]:
type(soup1)

bs4.BeautifulSoup

Get the number of total pages from

```
<nav class="paging js-paging" data-currentpage="1" data-totalpages="109">

```
Get items' info from
```
'''
<div class="product-card__content">
<p class="product-card__name"> Deanston 18 Year Old </p>
<p class="product-card__meta"> 70cl / 46.3% </p>
</div>
<div class="product-card__data">
<p class="product-card__price"> £74.95 </p>
<p class="product-card__unit-price"> (£107.07 per litre) </p>
</div>'''
```


In [ ]:
pages = int(soup1.find('nav', class_ = "paging js-paging").attrs['data-totalpages'])
pages

109

In [ ]:
prod_content = soup1.find_all('div', class_="product-card__content")
prod_data = soup1.find_all('div', class_="product-card__data")
# Should not continute put to the dataframe if not all items have both tags!
len(prod_content) == len(prod_data) 

True

In [ ]:
prod_data[0]

<div class="product-card__data"><p class="product-card__price"> £74.95 </p><p class="product-card__unit-price"> (£107.07 per litre) </p></div>

In [ ]:
i = 10
print(
    prod_content[i].find('p',class_="product-card__name").text.strip(),
    prod_content[i].find('p',class_="product-card__meta").text.strip(),
    prod_data[i].find('p',class_="product-card__price").text.strip().strip('£'),
    re.sub('[^\d\.]','', prod_data[i].find('p',class_="product-card__unit-price").text.strip()),
    prod_content[i].find('p',class_="product-card__meta").text.strip().split('/'),
    sep='\n'
)

Aberlour 200613 Year Old Sherry Cask Distillery Reserve Collection
50cl / 60.9%
89.95
179.90
['50cl ', ' 60.9%']


In [ ]:
df = pd.DataFrame(columns=['name', 'volumn(cl)', 'alcohol', 'price(£)', 'price/l'])
for i in range(len(prod_content)):
  templist = [
      prod_content[i].find('p',class_="product-card__name").text.strip(),
      float(prod_data[i].find('p',class_="product-card__price").text.strip().strip('£')),
      float(re.sub('[^\d\.]','', 
        prod_data[i].find('p',class_="product-card__unit-price").text.strip()))
      ]
  cardmeta = prod_content[i].find('p',class_="product-card__meta").text.strip().split('/')
  templist.insert(1,
      float(cardmeta[1].strip().strip('%')))
  templist.insert(1,
      int(cardmeta[0].strip().strip('cl')))
  tempdf = pd.DataFrame(
      [templist],
      columns=['name', 'volumn(cl)', 'alcohol', 'price(£)', 'price/l'])
  df = pd.concat([df,tempdf],ignore_index=True)

In [ ]:
df

,name,volumn(cl),alcohol,price(£),price/l
0,Deanston 18 Year Old,70,46.3,74.95,107.07
1,Lagavulin 16 Year Old,70,43.0,81.75,116.79
2,Balvenie 14 Year OldCaribbean Cask,70,43.0,59.95,85.64
3,Macallan 12 Year OldSherry Oak,70,40.0,79.95,114.21
4,Mortlach 15 Year OldGame of Thrones Six Kingdoms,70,46.0,84.95,121.36
5,Balvenie 21 Year OldSecond Red Rose Stories,70,48.1,350.00,500.00
6,Caperdonich 18 Year Old PeatedSecret Speyside,70,48.0,120.00,171.43
7,Macallan 12 Year Old Double Cask,70,40.0,66.95,95.64
8,Glendronach 15 Year Old RevivalSherry Cask,70,46.0,69.95,99.93
9,Arran 10 Year Old,70,46.0,38.75,55.36


In [ ]:
from sqlite3 import DataError
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

url = 'https://www.thewhiskyexchange.com/c/40/single-malt-scotch-whisky'

def soup(url:str) -> BeautifulSoup:
  return BeautifulSoup(requests.get(url).content,'lxml')

# transfer
def soup_to_df(soup) -> pd.DataFrame:
    prod_content = soup.find_all('div', class_="product-card__content")
    prod_data = soup.find_all('div', class_="product-card__data")
    # Should not continute put to the dataframe if not all items have both tags!
    if len(prod_content) != len(prod_data):
        raise Exception('the webpage content is missing something')
    
    df = pd.DataFrame(columns=['name', 'volumn(cl)', 'alcohol', 'price(£)', 'price/l'])
    for i in range(len(prod_content)):
        templist = [
            prod_content[i].find('p',class_="product-card__name").text.strip(),
            float(prod_data[i].find('p',class_="product-card__price").text.strip().strip('£')),
            float(re.sub('[^\d\.]','', 
                prod_data[i].find('p',class_="product-card__unit-price").text.strip()))
            ]
        cardmeta = prod_content[i].find('p',class_="product-card__meta").text.strip().split('/')
        templist.insert(1,
            float(cardmeta[1].strip().strip('%')))
        templist.insert(1,
            int(cardmeta[0].strip().strip('cl')))
        tempdf = pd.DataFrame(
            [templist],
            columns=['name', 'volumn(cl)', 'alcohol', 'price(£)', 'price/l'])
        df = pd.concat([df,tempdf],ignore_index=True)

    return df

soup = soup(url)

# get how many pages
pages = int(soup.find('nav', class_ = "paging js-paging").attrs['data-totalpages'])

#get data from the first(main) page
df = soup_to_df(soup)

#get data from the following pages
for page in range(pages):
    soup(url+'?pg='+str(page))
    df = pd.concat([df,soup_to_df(soup)],ignore_index=True)


In [ ]:
df.to_csv('wisky.csv')

In [ ]:
df = pd.read_csv('wisky1.csv')

SyntaxError: ignored

In [ ]:
for index, row in df.iterrows():
    break

In [ ]:
row['price(£)']

74.95